# Problem Set 1

Solutions to Computational Problems

## ATE under Exogeneity

---

# Problem 3

In [1]:
from collections.abc import Callable
from functools import partial

import numpy as np
import pandas as pd
from scipy.special import expit
from statsmodels.discrete.discrete_model import Probit

In [2]:
def _simulate_from_model(
    treatment_effect: float | Callable,
    n_samples: int,
    n_sim: int,
    seed: int,
):
    """Simulate data from the model.

    Args:
        treatment_effect (float | Callable): Treatment effect. Must be a float or a
            function that depends on the regressors.
        n_samples (int): Number of samples per simulation.
        n_sim (int): Number of simulations.
        seed (int): Random seed that is passed to jax rng.

    Returns:
        - np.ndarray: Outcomes of shape (n_sim, n_samples).
        - np.ndarray: Regressors of shape (n_sim, n_samples).
        - np.ndarray: Binary treatment status of shape (n_sim, n_samples).

    """
    rng = np.random.default_rng(seed)

    e = rng.normal(size=(n_sim, n_samples))
    x = rng.uniform(size=(n_sim, n_samples))

    propensity_score = expit(-(2 * x - 0.5))
    d = rng.binomial(1, propensity_score)

    te = treatment_effect(x) if callable(treatment_effect) else treatment_effect

    y = te * d + 0.5 * x + e

    return y, x, d

In [3]:
def _estimate_ate_via_naive_ols(y, x, d, with_intercept):
    """Estimate the ATE via the naive OLS estimator.

    This regresses y on (1, d, x) if with_intercept is True, or on (d, x) if
    with_intercept is False, and then takes the coefficient on d as the ATE.

    """
    # build feature matrix for model y ~ 1 + d + x or y ~ d + x
    if with_intercept:
        features = np.column_stack((np.ones_like(d), d, x))
    else:
        features = np.column_stack((d, x))

    coef = np.linalg.lstsq(features, y, rcond=None)[0]
    return coef[1] if with_intercept else coef[0]


def _estimate_ate_via_conditional_means(y, x, d):
    """Estimate the ATE via the conditional means estimator.

    This estimates the conditional mean of y given x for each treatment group, and then
    takes the difference between the two.

    """
    # build feature matrix for model y ~ 1 + x
    intercept = np.ones_like(x)
    features = np.column_stack((intercept, x))

    d_bool = d.astype(bool)

    # get least squares cofficients for mu_1 and mu_0
    coef_mu_1 = np.linalg.lstsq(features[d_bool], y[d_bool], rcond=None)[0]
    coef_mu_0 = np.linalg.lstsq(features[~d_bool], y[~d_bool], rcond=None)[0]

    # calculate mu_hat
    mu_hat_1 = features @ coef_mu_1
    mu_hat_0 = features @ coef_mu_0

    # calculate the ate as the mean difference between the conditional expectations
    return np.mean(mu_hat_1 - mu_hat_0, axis=0)


def _estimate_ate_via_propensity_score(y, x, d):
    """Estimate the ATE via the propensity score estimator for a single simulation.

    This fits a probit model to the treatment status d using x as a regressor, and then
    estimates the ATE using the inverse probability weighting formula.

    """
    # build feature matrix for simple probit model
    intercept = np.ones_like(x)
    features = np.column_stack((intercept, x))

    # estimate probit model
    model = Probit(
        endog=d,
        exog=features,
    )
    res = model.fit(maxiter=10, disp=False)

    # get predictions
    p_hat = res.predict(features)

    return np.mean(d * y / p_hat - (1 - d) * y / (1 - p_hat))


def estimate_ate(
    y,
    x,
    d,
    method,
):
    """Estimate the average treatment effect (ATE) given the data.

    Args:
        y (np.ndarray): Outcomes of shape (n_sim, n_samples).
        x (np.ndarray): Regressors of shape (n_sim, n_samples).
        d (np.ndarray): Binary treatment status of shape (n_sim, n_samples).
        method (str): Estimation method. Must be one of "naive_ols",
            "naive_ols_with_intercept", "conditional_means", or "propensity_score".

    Returns:
        float or np.ndarray: Estimated ATE. If y is 1D, returns a float. If y is 2D,
            returns a np.ndarray of shape (n_sim,).

    """
    estimation_methods = {
        "naive_ols": partial(_estimate_ate_via_naive_ols, with_intercept=False),
        "naive_ols_with_intercept": partial(
            _estimate_ate_via_naive_ols,
            with_intercept=True,
        ),
        "conditional_means": _estimate_ate_via_conditional_means,
        "propensity_score": _estimate_ate_via_propensity_score,
    }
    if method not in estimation_methods:
        raise ValueError(f"Invalid estimation method: {method}")

    estimation_func = estimation_methods[method]

    if y.ndim == 1:
        ate = estimation_func(y, x, d)
    elif y.ndim == 2:
        ate = np.array([estimation_func(y[i], x[i], d[i]) for i in range(y.shape[0])])
    else:
        raise ValueError(f"Invalid shape for y: {y.shape}")

    return ate

In [4]:
def simulation(n_samples, n_sim, seed=None):
    """Run a simulation.

    Args:
        n_samples (int): Number of samples per simulation.
        n_sim (int): Number of simulations.
        seed (int): Random seed.

    Returns:
        pd.DataFrame: Dataframe containing the results of the simulation.

    """
    if seed is None:
        seed = np.random.default_rng().integers(0, 2**32)

    treatment_effects = {
        "constant": 1.0,
        "function": lambda x: 2 * (x < 0.5),
    }

    res = pd.DataFrame(columns=["treatment_effect", "method"]).set_index(
        ["treatment_effect", "method"],
    )

    for te_name, te in treatment_effects.items():

        y, x, d = _simulate_from_model(
            treatment_effect=te,
            n_samples=n_samples,
            n_sim=n_sim,
            seed=seed,
        )

        for method in [
            "naive_ols",
            "naive_ols_with_intercept",
            "conditional_means",
            "propensity_score",
        ]:

            ate = estimate_ate(y, x, d, method=method)

            res.loc[(te_name, method), "sim_ate_mean"] = ate.mean()
            res.loc[(te_name, method), "sim_ate_var"] = ate.var()

    return res.sort_index()

In [5]:
res = simulation(n_samples=500, n_sim=1_000, seed=12345)

In [6]:
res  # noqa: B018

sim_ate_mean  sim_ate_var
treatment_effect method                                             
constant         conditional_means             1.000326     0.008763
                 naive_ols                     1.001274     0.006289
                 naive_ols_with_intercept      1.000079     0.008609
                 propensity_score              1.001124     0.008978
function         conditional_means             1.002250     0.011915
                 naive_ols                     1.422316     0.010003
                 naive_ols_with_intercept      1.115790     0.012529
                 propensity_score              0.998062     0.011688

## Instrumental Variables

--- 

## Problem 4

In [7]:
data = pd.read_feather("data/angrist_and_evans.feather")

> If you don't have the data stored as a feather file you can use
> ```python
> pd.read_excel("path/to/excel/file")
> ```

In [8]:
dependent_variables_info = {
    "workedm": "Worked for pay",
    "weeksm1": "Weeks worked",
    "hourswm": "Hours worked",
    "incomem": "Labor income",
    "famincl": "Log family income",
}

independent_variables = [
    "morekids",
    "agem1",
    "agefstm",
    "boy1st",
    "boy2nd",
    "blackm",
    "hispm",
    "othracem",
]

instrumental_mapping = {"morekids": "samesex"}

### Functions

In [9]:
def _cov_sandwich_estimator_ols(x: np.ndarray, e: np.ndarray) -> np.ndarray:
    r"""Estimator for the asymptotic covariance matrix of OLS estimator.

    The code corresponds to the HC0 estimator of $V_\hat{\beta}}$. For reference see
    Section 7.8 in Econometrics by Bruce Hansen (version January 29, 2021).

    Args:
        x (np.ndarray): Regressors, shape (n, p).
        e (np.ndarray): Residuals, shape (n,)

    Returns:
        np.ndarray: Asymptotic covariance matrix, shape (p, p)

    """
    xtx_inverse = np.linalg.pinv(x.T @ x)
    scaling = (x.T * e**2) @ x
    return xtx_inverse @ scaling @ xtx_inverse


def _cov_sandwich_estimator_iv(
    x: np.ndarray,
    z: np.ndarray,
    e: np.ndarray,
) -> np.ndarray:
    r"""Estimator for the asymptotic covariance matrix of 2SLS IV estimator.

    The code corresponds to the heteroskedasticity-robust estimator of $V_\\hat{\beta}}$
    For reference see Section 12.18 in Econometrics by Bruce Hansen (version January 29,
    2021).

    Args:
        x (np.ndarray): Regressors, shape (n, p).
        z (np.ndarray): Instruments, shape (n, p).
        e (np.ndarray): Residuals, shape (n,)

    Returns:
        np.ndarray: Asymptotic covariance matrix, shape (p, p)

    """
    ztz_inv = np.linalg.pinv(z.T @ z)
    ztx = z.T @ x
    xtz = ztx.T
    outer = np.linalg.pinv(xtz @ ztz_inv @ ztx) @ xtz @ ztz_inv
    scaling = (z.T * e**2) @ z
    return outer @ scaling @ outer.T


def _format_entries(coef: str, se: str) -> str:
    """Paste together two strings."""
    return f"{coef:.4f} ({se:.4f})"


def _format_frame(result_frame: pd.DataFrame, name: str) -> pd.DataFrame:
    """Format coef and se column to single.

    Args:
        result_frame (pd.DataFrame): Result frame; has columns only "coef" and "se".
        name (str): Name of the new column.

    Returns:
        pd.DataFrame: Formatted frame.

    """
    str_repr = [_format_entries(*row[1]) for row in result_frame.iterrows()]
    return pd.DataFrame(str_repr, index=result_frame.index, columns=[name])

### OLS

In [10]:
# get y and x from data


y = data[dependent_variables_info.keys()]

features = data[independent_variables]

# add intercept
x = np.column_stack([np.ones(len(data)), features])

In [11]:
# run least squares regression for each dependent variable

coef_ols, *_ = np.linalg.lstsq(x, y, rcond=None)

In [12]:
# store the result with correct index and column names

coef_ols = pd.DataFrame(
    coef_ols,
    index=["intercept", *independent_variables],
    columns=y.columns,
)

In [13]:
# compute standard errors for each dependent variable

residuals = y - x @ coef_ols

se_ols = {}

for outcome in dependent_variables_info:
    residual_array = residuals[outcome].to_numpy().flatten()

    cov = _cov_sandwich_estimator_ols(x, e=residual_array)

    se_ols[outcome] = pd.Series(np.sqrt(np.diag(cov)), index=coef_ols.index)


se_ols = pd.DataFrame(se_ols, columns=y.columns)

In [14]:
# combine coefficients and standard error in one frame

result_ols = pd.concat(
    [coef_ols.loc["morekids"], se_ols.loc["morekids"]],
    axis=1,
    keys=["coef", "se"],
)

result_ols = result_ols.rename(mapper=dependent_variables_info, axis=0)

result_ols = _format_frame(result_ols, name="ols")

## IV 

In [15]:
# get z from data (y and x are already defined)

instruments_names = [
    instrumental_mapping.get(var, var) for var in independent_variables
]

instruments = data[instruments_names]

# add intercept
z = np.column_stack([np.ones(len(data)), instruments])

In [16]:
# run two-stage least squares regression

# first stage

first_stage_coef, *_ = np.linalg.lstsq(z, x, rcond=None)
x_predicted = z @ first_stage_coef

# second stage

coef_iv, *_ = np.linalg.lstsq(x_predicted, y, rcond=None)

In [17]:
# store the result with the correct index and column names

coef_iv = pd.DataFrame(
    coef_iv,
    index=["intercept", *independent_variables],
    columns=y.columns,
)

In [18]:
# compute standard errors for each dependent variable

residuals = y - x @ coef_ols

se_iv = {}

for outcome in dependent_variables_info:
    residual_array = residuals[outcome].to_numpy().flatten()

    cov = _cov_sandwich_estimator_iv(x, z=z, e=residual_array)

    se_iv[outcome] = pd.Series(np.sqrt(np.diag(cov)), index=coef_ols.index)


se_iv = pd.DataFrame(se_iv, columns=y.columns)

In [19]:
# combine coefficients and standard error in one frame

result_iv = pd.concat(
    [coef_iv.loc["morekids"], se_iv.loc["morekids"]],
    axis=1,
    keys=["coef", "se"],
)

result_iv = result_iv.rename(mapper=dependent_variables_info, axis=0)

result_iv = _format_frame(result_iv, name="iv")

### Result

In [20]:
_result = pd.concat([result_ols, result_iv], axis=1)

In [21]:
_result  # noqa: B018

,ols,iv
Worked for pay,-0.1764 (0.0016),-0.1173 (0.0251)
Weeks worked,-8.9782 (0.0706),-5.5588 (1.1147)
Hours worked,-6.6467 (0.0610),-4.5468 (0.9523)
Labor income,-3762.3826 (34.4127),-1902.9526 (544.5210)
Log family income,-0.1379 (0.0045),-0.0253 (0.0683)
